<a href="https://colab.research.google.com/github/Balonglongz/github-codespaces-demo/blob/main/PFX_Fall22_SkillsOH_78_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `Final Exam`, `Fall 2022`: `Time Series Analysis of US Inflation`
_Version 1.0.1_

Change history:   
1.0.1 - bugfix ex2 test code.  
1.0 - initial release  

*All of the header information is important. Please read it..*

**Topics, number of exercises:** This problem builds on your knowledge of Pandas, Numpy, basic Python data structures, and implementing mathematical functions. It has **9** exercises, numbered 0 to **8**. There are **18** available points. However, to earn 100% the threshold is **13** points. (Therefore, once you hit **13** points, you can stop. There is no extra credit for exceeding this threshold.)

**Exercise ordering:** Each exercise builds logically on previous exercises, but you may solve them in any order. That is, if you can't solve an exercise, you can still move on and try the next one. Use this to your advantage, as the exercises are **not** necessarily ordered in terms of difficulty. Higher point values generally indicate more difficult exercises.

**Demo cells:** Code cells starting with the comment `### define demo inputs` load results from prior exercises applied to the entire data set and use those to build demo inputs. These must be run for subsequent demos to work properly, but they do not affect the test cells. The data loaded in these cells may be rather large (at least in terms of human readability). You are free to print or otherwise use Python to explore them, but we did not print them in the starter code.

**Debugging your code:** Right before each exercise test cell, there is a block of text explaining the variables available to you for debugging. You may use these to test your code and can print/display them as needed (careful when printing large objects, you may want to print the head or chunks of rows at a time).

**Exercise point breakdown:**

- Exercise 0: **1** point(s)
- Exercise 1: **1** point(s)
- Exercise 2: **2** point(s)
- Exercise 3: **2** point(s)
- Exercise 4: **2** point(s)
- Exercise 5: **2** point(s)
- Exercise 6: **2** point(s)
- Exercise 7: **3** point(s)
- Exercise 8: **3** point(s)

**Final reminders:**

- Submit after **every exercise**
- Review the generated grade report after you submit to see what errors were returned
- Stay calm, skip problems as needed, and take short breaks at your leisure


## Background Inflation

Inflation is an increase in overall prices in an economy over time. Deflation is "negative inflation", a decrease in prices over time. A common way to measure inflation is to first calculate the CPI (price of a representative basket of goods), then compute the difference in CPI over a time interval. In other words if the CPI is 100 at one point in time, and the CPI is 105 one year later then we would say that the inflation rate over that year was 5%.

## Data

We have obtained the US CPI for each month going back to the early 20th century from The Organisation for Economic Co-operation and Development.

## Analysis goals
- Use the CPI data to calculate the inflation rate at any point in history over an arbitrary number of months.
- Attempt to predict the inflation rate in future months based on the inflation rate in previous months using exponential smoothing models.
    - Evaluate how "good" the predictions are.
    - Tune the models to pick the best parameters.
    - Make inferences based on the selected parameters.

In [1]:
# uncomment in Google Colab
# !python --version
!pip install dill
import dill as pickle
!pip install cryptography

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.5 MB/s eta 0:00:00


In [2]:
### Global Imports
import pandas as pd
import numpy as np
import pickle

# Some functionality needed by the notebook and demo cells:
from pprint import pprint, pformat
import math

# === Messages === #

def status_msg(s, verbose=True, **kwargs):
    if verbose:
        print(s, **kwargs)

# === Input/output === #

# def load_df_from_file(basename, dirname='resource/asnlib/publicdata/', abort_on_error=False, verbose=False):
def load_df_from_file(basename, dirname='', abort_on_error=False, verbose=False):
    from os.path import isfile
    from dill import loads
    from pandas import DataFrame
    df = DataFrame()
    filename = f"{dirname}{basename}"
    status_msg(f"Loading `DataFrame` from '{filename}'...", verbose=verbose)
    if isfile(filename):
        try:
            with open(filename, "rb") as fp:
                df = loads(fp.read())
            status_msg(f"  ==> Done!", verbose=verbose)
        except:
            if abort_on_error:
                raise
            else:
                df = DataFrame()
                status_msg(f"  ==> An error occurred.", verbose=verbose)
    return df

# def load_obj_from_file(basename, dirname='resource/asnlib/publicdata/', abort_on_error=False, verbose=False):
def load_obj_from_file(basename, dirname='', abort_on_error=False, verbose=False):
    from os.path import isfile
    from dill import loads
    from pandas import DataFrame
    filename = f"{dirname}{basename}"
    status_msg(f"Loading object from '{filename}'...", verbose=verbose)
    if isfile(filename):
        try:
            with open(filename, "rb") as fp:
                df = loads(fp.read())
            status_msg(f"  ==> Done! Type: `{type(df)}`", verbose=verbose)
        except:
            if abort_on_error:
                raise
            else:
                df = DataFrame()
                status_msg(f"  ==> An error occurred.", verbose=verbose)
    else:
        df = None
    return df

In [3]:
# import files
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_12_FEX_FA22_78/main/tc_7
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_12_FEX_FA22_78/main/tc_8
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_12_FEX_FA22_78/main/cpi_urban_all.csv

!mkdir tester_fw
%cd tester_fw

!wget https://raw.githubusercontent.com/gt-cse-6040/topic_12_FEX_FA22_78/main/tester_fw/__init__.py
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_12_FEX_FA22_78/main/tester_fw/test_utils.py
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_12_FEX_FA22_78/main/tester_fw/testers.py

%cd ..

--2023-11-22 03:32:27--  https://raw.githubusercontent.com/gt-cse-6040/topic_12_FEX_FA22_78/main/tc_7
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54500 (53K) [text/plain]
Saving to: ‘tc_7’

tc_7                100%[===================>]  53.22K  --.-KB/s    in 0.01s   

2023-11-22 03:32:27 (3.58 MB/s) - ‘tc_7’ saved [54500/54500]

--2023-11-22 03:32:27--  https://raw.githubusercontent.com/gt-cse-6040/topic_12_FEX_FA22_78/main/tc_8
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 909880 (889K) [text/plain]
Saving to: ‘tc_8’

tc_

## Exercise 0 - (**1** Points):
To start things off we will load the CPI data into the notebook environment. You do not need to modify the cell below, just execute the test and collect your free point!

This cell will also display the first few rows and last few rows of the CPI data we just loaded.

In [4]:
cpi_all_df = pd.read_csv('cpi_urban_all.csv')
display(cpi_all_df.head())
display(cpi_all_df.tail())

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,HALF1,HALF2
0,1913,9.8,9.8,9.8,9.8,9.7,9.8,9.9,9.9,10.0,10.0,10.1,10.0,,
1,1914,10.0,9.9,9.9,9.8,9.9,9.9,10.0,10.2,10.2,10.1,10.2,10.1,,
2,1915,10.1,10.0,9.9,10.0,10.1,10.1,10.1,10.1,10.1,10.2,10.3,10.3,,
3,1916,10.4,10.4,10.5,10.6,10.7,10.8,10.8,10.9,11.1,11.3,11.5,11.6,,
4,1917,11.7,12.0,12.0,12.6,12.8,13.0,12.8,13.0,13.3,13.5,13.5,13.7,,


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,HALF1,HALF2
105,2018,247.867,248.991,249.554,250.546,251.588,251.989,252.006,252.146,252.439,252.885,252.038,251.233,250.089,252.125
106,2019,251.712,252.776,254.202,255.548,256.092,256.143,256.571,256.558,256.759,257.346,257.208,256.974,254.412,256.903
107,2020,257.971,258.678,258.115,256.389,256.394,257.797,259.101,259.918,260.280,260.388,260.229,260.474,257.557,260.065
108,2021,261.582,263.014,264.877,267.054,269.195,271.696,273.003,273.567,274.310,276.589,277.948,278.802,266.236,275.703
109,2022,281.148,283.716,287.504,289.109,292.296,296.311,296.276,296.171,296.808,298.012,,,288.347,


<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 0. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [5]:
### test_cell_ex0
assert 'cpi_all_df' in globals()
assert isinstance(cpi_all_df, pd.DataFrame)
print('Passed! Please submit.')

Passed! Please submit.


## Grid search

Calculus won't work for finding the optimal parameters for either flavor of exponential smoothing implemented above. A "brute-force" alternative is to generate a list of suitable candidates for each parameter and test our functions on all possible combinations. This is called a grid search. It's not flashy, but for a lot of modeling techniques it's one of the only suitable choices.

## Exercise 7 - (**3** Points):
Define the function `build_grid(params)`. The input `params` will be a dictionary mapping strings to arrays (or array-like data structures such as `list`s). Consider each string a parameter name and each value in the corresponding array to be a candidate value for that parameter.

Your function should return a list of dictionaries which satisfies the following:  
- Each dictionary maps each parameter name to exactly one candidate value for that parameter.
- Exactly one dictionary exists for each combination of parameters.
- The list is sorted by the values associated with each key. The sort priority for each key should be based on their lexographical order.

For example, `build_params({'b': [1, 2], 'z': [3, 5, 6], 'a': [100, 10]})` should return:  
```
[{'b': 1, 'z': 3, 'a': 10},
 {'b': 1, 'z': 5, 'a': 10},
 {'b': 1, 'z': 6, 'a': 10},
 {'b': 2, 'z': 3, 'a': 10},
 {'b': 2, 'z': 5, 'a': 10},
 {'b': 2, 'z': 6, 'a': 10},
 {'b': 1, 'z': 3, 'a': 100},
 {'b': 1, 'z': 5, 'a': 100},
 {'b': 1, 'z': 6, 'a': 100},
 {'b': 2, 'z': 3, 'a': 100},
 {'b': 2, 'z': 5, 'a': 100},
 {'b': 2, 'z': 6, 'a': 100}]
```

Notice the following:  
- There are 3 parameters. The length of the lists assigned to the parameters are 2, 3, and 2.  
    - Thus there are 12 (`2*3*2`)possible combinations.  
- Each possible combination is represented exactly once in the list.
- The list is sorted first by the `'a'` value, then by the `'b'` value, and finally by the `'z'` value.

Keep in mind that the function needs to work for an **arbitrary** dictionary (the number of keys, the keys themselves, and the values can be **anything** which meets the which has the structure given earlier in the prompt).

**Note**: You may find the functions `itertools.product` or `numpy.meshgrid` helpful in solving this problem.

In [11]:
### Define demo inputs

demo_params_ex7 = {'b': [1, 2], 'z': [3, 5, 6], 'a': [100, 10]}

In [7]:
### Exercise 7 solution
def build_grid(params):
    ### BEGIN SOLUTION
    keys = sorted(params.keys())
    grid = np.meshgrid(*params.values())
    grid = [ar.reshape(np.prod(ar.shape),1) for ar in grid]
    grid_arr = np.concatenate(grid, axis=1)
    param_dicts = [{k: row[i] for i, k in enumerate(params.keys())} for row in grid_arr]
    return sorted(param_dicts, key=lambda d: tuple(d[k] for k in keys))
    ### END SOLUTION

### demo function call
build_grid(demo_params_ex7)

[{'b': 1, 'z': 3, 'a': 10},
 {'b': 1, 'z': 5, 'a': 10},
 {'b': 1, 'z': 6, 'a': 10},
 {'b': 2, 'z': 3, 'a': 10},
 {'b': 2, 'z': 5, 'a': 10},
 {'b': 2, 'z': 6, 'a': 10},
 {'b': 1, 'z': 3, 'a': 100},
 {'b': 1, 'z': 5, 'a': 100},
 {'b': 1, 'z': 6, 'a': 100},
 {'b': 2, 'z': 3, 'a': 100},
 {'b': 2, 'z': 5, 'a': 100},
 {'b': 2, 'z': 6, 'a': 100}]

In [19]:
from itertools import product

def build_grid(params):
    # Define the custom order in which keys should be sorted
    custom_order = ['b', 'z', 'a']

    # Create a new list for keys sorted according to custom_order
    sorted_keys = [key for key in custom_order if key in params]

    # Create a list to store all combinations of parameter values
    combinations = list(product(*(params[key] for key in sorted_keys)))

    # Create a list to store the dictionaries for each combination
    grid = []
    for combination in combinations:
        combination_dict = {sorted_keys[i]: combination[i] for i in range(len(sorted_keys))}
        grid.append(combination_dict)

    # Define a function for sorting dictionaries according to custom order
    def sort_key(dictionary):
        return tuple(dictionary[key] for key in custom_order if key in dictionary)

    # Sort the grid based on the values in each dictionary according to custom order
    grid.sort(key=sort_key)

    return grid

### demo function call
build_grid(demo_params_ex7)

[{'b': 1, 'z': 3, 'a': 10},
 {'b': 1, 'z': 3, 'a': 100},
 {'b': 1, 'z': 5, 'a': 10},
 {'b': 1, 'z': 5, 'a': 100},
 {'b': 1, 'z': 6, 'a': 10},
 {'b': 1, 'z': 6, 'a': 100},
 {'b': 2, 'z': 3, 'a': 10},
 {'b': 2, 'z': 3, 'a': 100},
 {'b': 2, 'z': 5, 'a': 10},
 {'b': 2, 'z': 5, 'a': 100},
 {'b': 2, 'z': 6, 'a': 10},
 {'b': 2, 'z': 6, 'a': 100}]

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 7. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [15]:
### test_cell_ex7
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_7',
    'func': build_grid, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'params':{
            'dtype':'dict', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'z0BNF11iKYQicR63590bVXZGa19YGvJcmzrbP6R7oAY=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


## A metric to compare models
In order to implement a grid search we need some metric of evaluating how well each model fits the data. We have provided `mse` to calculate the mean squared error for the predictions coming from a model. It takes two 1-D array arguments, `obs` (observed values) and `preds` (predicted values). It returns the mean of the squared difference between observations and predictions. A lower output from `mse` indicates that the model is a better fit.

In [ ]:
def mse(obs, preds):
    r = obs-preds
    return np.power(r,2).mean()

## Exercise 8 - (**3** Points):
Now that we have built our parameter grid, it's time to test it.

Define the function `grid_search(func, ts, grid, n_back)`. The inputs are as follows.  
- `func`: an _arbitrary_ function similar to the ones which we defined in exercises 5 and 6. It takes a 1-D time-series array and some additional parameters as inputs. It returns predictions for the time series based on some logic of which we are unaware.
    - You can consider `func(...)[i]` to be the prediction that corresponds with `ts[i]`.
    - `func(...)[-1]` is the prediction for the next value of `ts` which has not been observed yet.
- `ts`: a 1-D numerical array which is a suitable input for `func`. Think of this as time-series data that we want to fit some model defined by `func`.
- `grid`: a list of dictionaries. Each dictionary maps the remaining parameters for `func` to values. You can assume that the keys in these dictionaries match the remaining named parameters for `func`.  
- `n_back`: a positive integer smaller than `ts.shape[0]`.

Your function should iteratively search the parameter sets given in `grid` to determine the set which results in the best model (the one with the lowest `mse`) given `func` and `ts`. Since exponential smoothing takes time to ramp up, you should only consider the last `n_back` **observations** in `mse` calculations. If there are multiple parameter sets which result in the same `mse` the parameter set encountered _first_ in `grid` should be chosen.

You can follow this whiteboard-level algorithm to implement your search.  
1. Initialize a variable to track the lowest `mse` returned so far in the search.
2. Initialize a variable to track the best set of parameters tested so far.
3. Iterate through each `dict` (we will call the dict `params`) in `grid`.
    - Calculate the predictions for this set of parameters.
        - `func(ts, **params)` will calculate the predictions.
    - Slice the predictions and observations so that only data points corresponding to the last `n_back` **observations** are in the slices.
    - Calculate the `mse` of the slices.
    - Update the tracking variables set in steps 1 and 3 if the `mse` has improved.
4. Once all `dict`s in `grid` have been checked, return the best set of parameters.


In [ ]:
### Define demo inputs
def demo_func_ex8(ts, a, b):
    return np.concatenate(( np.array([np.nan]),
                            ((5*a)/(7*b))*ts[1:],
                            np.array([50.])))
demo_ts_ex8 = np.array([51.1, 61.7, 34.92, 7.97, 84.03, 29.65, 85.86, 95.4, 82., 36.61])
demo_grid_ex8 = [   {'a': 2, 'b': 5},
                    {'a': 2, 'b': 11},
                    {'a': 3, 'b': 5},
                    {'a': 3, 'b': 11},
                    {'a': 21, 'b': 15},
                    {'a': 7, 'b': 11},
                    {'a': 14, 'b': 10}]
demo_n_back_ex8 = 9

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
{'a': 21, 'b': 15}
```
Notice:  
- `func` will return "predictions" $\hat{x}_t = \frac{5a}{7b}x_t$ whenever $0 < t \le 9$.
- Due to the `n_back` parameter we are only looking at the last 9 observations (same interval as in the above point).
- When $a=7k$ and $b=5k$ with any number $k$, the fraction cancels and we have $\hat{x}_t = x_t$. This will have MSE of 0.
    - Both `{'a': 21, 'b': 15}` and `{'a': 14, 'b': 10}` match this form, so we choose `{'a': 21, 'b': 15}` which appeared first in `grid`.

In [ ]:
### Exercise 8 solution
def grid_search(func, ts, grid, n_back):
    ### BEGIN SOLUTION
    min_mse = np.inf
    best_params = {}
    for params in grid:
        preds = func(ts, **params)
        preds = preds[-(n_back+1):-1]
        obs = ts[-n_back:]
        cur_mse = mse(obs, preds)
        if cur_mse < min_mse:
            min_mse = cur_mse
            best_params = params
    return best_params
    ### END SOLUTION

### demo function call
grid_search(func=demo_func_ex8, ts=demo_ts_ex8, grid=demo_grid_ex8, n_back=9)

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 8. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex8
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_8',
    'func': grid_search, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'func':{
            'dtype':'function', # data type of param.
            'check_modified':False,
        },
        'ts':{
            'dtype':'np.ndarray', # data type of param.
            'check_modified':True,
        },
        'grid':{
            'dtype':'dict', # data type of param.
            'check_modified':True,
        },
        'n_back':{
            'dtype':'int', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'dict',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'z0BNF11iKYQicR63590bVXZGa19YGvJcmzrbP6R7oAY=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

**Fin.** If you have made it this far, congratulations on completing the exam. **Don't forget to submit!**